### 导入数据集

In [1]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

## 定义FP树的数据结构

In [2]:
class treeNode:
    def __init__(self, nameValue, numOccur,parentNode):
        self.name = nameValue        #节点名称                         
        self.count = numOccur        #计数值
        self.nodeLink = None         #链接相似的元素项
        self.parent = parentNode     #指向父节点
        self.children = {}           #指向子节点
    
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print(' '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [3]:
# 调试运行
rootNode = treeNode('pyramid', 9, None)
rootNode.children['eye'] = treeNode('eye', 13, None)
rootNode.children['phoenix'] = treeNode('phoenix', 3, None)
rootNode.disp()

  pyramid   9
   eye   13
   phoenix   3


## 构建FP树
* 定义头指针表：使用字典记录FP树中每类元素的总数并指向给定类型的第一个实例，可快速访问FP树中一个给定类型的所有元素。
* 第一次遍历，获得每个元素项的出现频率，去掉不满足最小支持度的元素项，将其保存到头指针表中。
* 对事务进行筛选、排序
* 构建FP树
    * 对每个项集将其添加到树中
        * 若路径存在，则增加已有的元素的值
        * 如路径不存在，则创建一条新的路径。

In [4]:
def updateHeader(nodeToTest, targetNode):
    '''
    更新头链表，确保节点链接指向该元素项的每一个实例
    输入：头指针列表中的指针nodeToTest，待链接的新节点targetNode
    '''
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodelink
    nodeToTest.nodelink = targetNode

In [5]:
def updateTree(items, inTree, headerTable,count):
    '''
    更新树，使树生长
    输入：元素排序后的事务items，待更新的FP树inTree，头指针列表headerTable, 事务出现的次数count
    '''
    if items[0] in inTree.children:                 # 若事务中的元素已经在树中，则计数值增加
        inTree.children[items[0]].inc(count)         
    else:                                           # 否则，新建一条分支
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:        # 如果头指针列表中没有指向该元素的指针，则添加
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:                                       #否则，更新头链表
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
            
    if len(items) > 1:                              # 迭代处理其他元素 
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [6]:
def createTree(dataSet, minSup=1):
    '''
    构建FP树
    输入：数据集dataSet，最小支持度minSup
    输出：FP树retTree，头指针列表headerTable
    '''
    
    headerTable = {}                         # 定义一个空字典，用于存储头指针列表
    for trans in dataSet:                    # 对数据集中的每一个事务                  #第一次遍历
        for item in trans:                   # 对 事务 中的每一个元素
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]  # 记录元素，并统计出现次数
    
    for k in list(headerTable.keys()):       # 去除不满足最小支持度的项集，得到只含频繁项集的头指针列表   
        if headerTable[k] < minSup:
            del(headerTable[k])
    
    freqItemSet = set(headerTable.keys())    # 保存频繁向集中的元素 
    #print(freqItemSet)
    if len(freqItemSet) == 0:                # 若频繁项集中没有元素，则退出       
        return None, None
    for k in freqItemSet:                    # 否则，对头指针列表进行修改，使之可以同时保存 元素出现的次数 和 指向每种类型第一个元素的指针
        headerTable[k] = [headerTable[k], None]
        
    retTree = treeNode('Null Set', 1, None)  # 初始化FP树
    
    for tranSet, count in dataSet.items():   # 根据全局频率对每个事务中的元素进行排序
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(),
                                                 key=lambda p:p[1], 
                                                 reverse=True)]
            #print(orderedItems)
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable  

### 测试构建的FP树

In [7]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [8]:
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [9]:
myFPtree, myHeaderTable = createTree(initSet, 3)
myFPtree.disp()

  Null Set   1
   z   5
    r   1
    x   3
     t   3
      y   3
       s   2
       r   1
   x   1
    s   1
     r   1


## 挖掘频繁项集
* 类似Apriori算法，先从单元素项开始，逐步构建更大的元素集合
* 从FP树中抽取频繁项集的三个基本步骤
    * 从Fp树中获得条件模式基
    * 利用条件模式基，构建一个条件FP树
    * 迭代重复上述两步，直到树包含一个元素项为止。

### 抽取条件模式基
条件模式基：以所查找元素项为结尾的路径集合。每条路径都是一条前缀路径，即介于所查找元素项与根节点之间的所有内容。
每一条前缀路径都与一个计数值关联，该计数值等于起始元素项的计数值。
获取前缀路径的方法：
* 对树进行穷举式搜索，直到获得想要的频繁项集为止
* 利用头指针表。头指针表包含相同类型元素链表的起始指针，一旦到达了每一个元素项，就可以上溯这棵树直到根节点为止。

In [10]:
def ascendTree(leafNode, prefixPath):   
    '''
    迭代上溯整棵树
    输入：叶子节点leafNode, 空的前缀路径列表prefixPath
    '''
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [11]:
def findPrefixPath(treeNode):
    '''
    由头指针表找到前缀路径
    输入：指定的树节点
    '''
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [12]:
myHeaderTable

{'z': [5, <__main__.treeNode at 0x4df6cc0>],
 'r': [3, <__main__.treeNode at 0x4df67f0>],
 't': [3, <__main__.treeNode at 0x4df6e10>],
 'y': [3, <__main__.treeNode at 0x4df6e80>],
 'x': [4, <__main__.treeNode at 0x4df6898>],
 's': [3, <__main__.treeNode at 0x4df69b0>]}

In [13]:
# 调试
findPrefixPath(myHeaderTable['x'][1])

{frozenset({'z'}): 3}

In [14]:
findPrefixPath(myHeaderTable['z'][1])

{}

In [15]:
findPrefixPath(myHeaderTable['y'][1])

{frozenset({'t', 'x', 'z'}): 3}

### 创建条件FP树
对于每一个频繁项集，都要创建一颗条件FP树

In [16]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[0])]   #对头指针列表中的元素项按照其出现频率进行排序
    
    for basePat in bigL:                          # 将每一项添加到频繁项集列表中
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        
        condPattBases = findPrefixPath(headerTable[basePat][1]) # 创建条件基
        #print(condPattBases)
       
        myCondTree, myHead = createTree(condPattBases, minSup)
        
        if myHead != None: #3. mine cond. FP-tree
            print('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [17]:
mineTree(myFPtree, myHeaderTable, 3, set([]),[])

conditional tree for:  {'t'}
  Null Set   1
   x   3
    z   3
conditional tree for:  {'t', 'z'}
  Null Set   1
   x   3
conditional tree for:  {'x'}
  Null Set   1
   z   3
conditional tree for:  {'y'}
  Null Set   1
   t   3
    x   3
     z   3
conditional tree for:  {'x', 'y'}
  Null Set   1
   t   3
conditional tree for:  {'y', 'z'}
  Null Set   1
   t   3
    x   3
conditional tree for:  {'x', 'y', 'z'}
  Null Set   1
   t   3


## 从新闻网站点击流中挖掘数据

In [18]:
parseDat = [line.split() for line in open('data/kosarak.dat').readlines()]
initSet = createInitSet(parseDat)
myFPtree, myHeaderTable = createTree(initSet, 100000)
myFreqList = []
mineTree(myFPtree, myHeaderTable, 100000, set([]), myFreqList)

conditional tree for:  {'11'}
  Null Set   1
   6   261773


In [20]:
len(myFreqList)

5

In [21]:
myFreqList

[{'1'}, {'11'}, {'11', '6'}, {'3'}, {'6'}]